In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.preprocessing import StandardScaler as sscaler
import importlib
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
import optuna

%matplotlib inline

In [33]:
df = pd.read_csv("train_meanprice.csv")
df_test = pd.read_csv("test_meanprice.csv")


In [34]:
n = np.array(df["n"]).reshape([-1, 1])

In [35]:

sc = sscaler()
sc.fit(n)
n = sc.transform(n)
df["n"] = n

In [36]:
df_train, df_val = train_test_split(df, test_size=0.2, shuffle=True, random_state=123)
    
col = "n"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

stacking_answer= df_val[col].to_list()

categorical_features = [
                    "Month",
                    "shopID",
                    "Cat",
                    "itemID"
                    ]

In [37]:

# OptunaによるlightGBMの最適パラメータ探索

def objective(trial):

    lgb_trains = lgb.Dataset(train_x, train_y)
    lgb_valids = lgb.Dataset(val_x, val_y)
    
    learning_rate = trial.suggest_float('learning_rate', 0.0, 1.0)
    num_leaves =  trial.suggest_int("num_leaves", 2, 100)
    tree_learner = trial.suggest_categorical('tree_learner', ["serial", "feature", "data", "voting"])
    lambda_l1= trial.suggest_float("lambda_l1", 0.0, 200.0)
    lambda_l2 = trial.suggest_float("lambda_l2", 0.0, 200.0)
    max_depth = trial.suggest_int("max_depth", 2, 12)
    num_iterations = trial.suggest_int("num_iterations", 10, 100)

    lgb_params = {
        "objective": "regression",
        "boosting_type": "gbdt", 
        "metrics": "rmse", 
        "learning_rate": learning_rate, 
        "num_leaves": num_leaves, 
        "tree_learner": tree_learner,
        "lambda_l1": lambda_l1, 
        "lambda_l2": lambda_l2, 
        "seed": 123, 
        "max_depth": max_depth,
        "num_iterations": num_iterations      
    }

    regressor = lgb.train(lgb_params,
                        lgb_trains,
                        valid_sets=lgb_valids, 
                        categorical_feature=categorical_features,
                        )

    lgb_pred_val_y = regressor.predict(val_x)

    rmse = np.sqrt(mse(val_y, lgb_pred_val_y))
    
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)


[I 2023-09-10 18:44:22,417] A new study created in memory with name: no-name-3be7a84d-3ed4-473e-a8b0-2b4fef4dffd4
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001740 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 458
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 5
[LightGBM] [Info] Start training from score -0.000149
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[I 2023-09-10 18:44:24,232] Trial 0 finished with value: 0.674507646900261 and parameters: {'learning_rate': 0.2331294884030286, 'num_leaves': 59, 'tree_learner': 'data', 'lambda_l1': 67.24149797193395, 'lambda_l2': 138.51860767482634, 'max_depth': 10, 'num_iterations': 52}. Best is trial 0 with value: 0.674507646900261.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000591 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 458
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 5
[LightGBM] [Info] Start training from score -0.000149
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[I 2023-09-10 18:44:25,391] Trial 1 finished with value: 0.7019836370531749 and parameters: {'learning_rate': 0.35523266685319, 'num_leaves': 32, 'tree_learner': 'data', 'lambda_l1': 147.4713427988617, 'lambda_l2': 162.00215614576717, 'max_depth': 7, 'num_iterations': 86}. Best is trial 0 with value: 0.674507646900261.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2023-09-10 18:44:25,573] Trial 2 finished with value: 0.769527328137685 and parameters: {'learning_rate': 0.26234194023911717, 'num_leaves': 6, 'tree_learner': 'feature', 'lambda_l1': 63.76816027026606, 'lambda_l2': 186.57874712946952, 'max_depth': 9, 'num_iterations': 26}. Best is trial 0 with value: 0.674507646900261.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:17

...

[I 2023-09-10 18:51:13,212] Trial 197 finished with value: 0.6317052641856942 and parameters: {'learning_rate': 0.58133622076997, 'num_leaves': 58, 'tree_learner': 'serial', 'lambda_l1': 8.508652931965687, 'lambda_l2': 64.91412142964649, 'max_depth': 7, 'num_iterations': 94}. Best is trial 166 with value: 0.6218594899243519.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001193 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 458
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 5
[LightGBM] [Info] Start training from score -0.000149
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[I 2023-09-10 18:51:15,490] Trial 198 finished with value: 0.6360629747266116 and parameters: {'learning_rate': 0.5194416261425193, 'num_leaves': 82, 'tree_learner': 'serial', 'lambda_l1': 3.617498555141616, 'lambda_l2': 77.37107992764676, 'max_depth': 7, 'num_iterations': 56}. Best is trial 166 with value: 0.6218594899243519.
/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000319 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 458
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 5
[LightGBM] [Info] Start training from score -0.000149


[I 2023-09-10 18:51:17,744] Trial 199 finished with value: 0.6427536335909692 and parameters: {'learning_rate': 0.8331183040203405, 'num_leaves': 44, 'tree_learner': 'serial', 'lambda_l1': 13.508198881672966, 'lambda_l2': 71.58748742592077, 'max_depth': 8, 'num_iterations': 58}. Best is trial 166 with value: 0.6218594899243519.


In [38]:

# パラメータ最適化したlightGBMによる予想

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

model_lgb = lgb.train(study.best_trial.params, 
                    trains, 
                    valid_sets=valids, 
                    categorical_feature=categorical_features
                    )

val_lgb = model_lgb.predict(val_x)
rmse = np.sqrt(mse(val_y, val_lgb))

lgb_pred = model_lgb.predict(df_test)

rmse


/Users/makoto/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000975 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 458
[LightGBM] [Info] Number of data points in the train set: 53856, number of used features: 5
[LightGBM] [Info] Start training from score -0.000149
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

0.6218594899243519

In [56]:
lgb_pred

array([ 0.45954135,  0.03510352,  0.12765504, ..., -0.2903344 ,
       -0.48265343, -0.27556351])

In [58]:
submit = np.array(lgb_pred).reshape([-1, 1])
submit = sc.inverse_transform(n)
submit = pd.DataFrame(submit)


In [60]:
submit.to_csv("submit27.csv", header=False)


In [41]:
'''
def objective(trial):
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    model = cb.CatBoostRegressor(**params, silent=True)
    model.fit(train_x, train_y)
    predictions = model.predict(val_x)
    rmse = mse(val_y, predictions, squared=False)
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
'''

'\ndef objective(trial):\n    params = {\n        "iterations": 1000,\n        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),\n        "depth": trial.suggest_int("depth", 1, 10),\n        "subsample": trial.suggest_float("subsample", 0.05, 1.0),\n        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),\n        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),\n    }\n\n    model = cb.CatBoostRegressor(**params, silent=True)\n    model.fit(train_x, train_y)\n    predictions = model.predict(val_x)\n    rmse = mse(val_y, predictions, squared=False)\n    return rmse\n\nstudy = optuna.create_study(direction=\'minimize\')\nstudy.optimize(objective, n_trials=100)\n'

In [42]:
'''
#Catboostによる予想

cb_train = cb.Pool(train_x, label=train_y, cat_features=categorical_features)
cb_test = cb.Pool(val_x, label=val_y, cat_features=categorical_features)

model_cb = cb.CatBoostRegressor(**study.best_params, loss_function='RMSE')
model_cb.fit(cb_train)

val_cb = model_cb.predict(val_x)
rmse = np.sqrt(mse(val_y, val_cb))

cb_pred = model_cb.predict(df_test)

rmse
'''


"\n#Catboostによる予想\n\ncb_train = cb.Pool(train_x, label=train_y, cat_features=categorical_features)\ncb_test = cb.Pool(val_x, label=val_y, cat_features=categorical_features)\n\nmodel_cb = cb.CatBoostRegressor(**study.best_params, loss_function='RMSE')\nmodel_cb.fit(cb_train)\n\nval_cb = model_cb.predict(val_x)\nrmse = np.sqrt(mse(val_y, val_cb))\n\ncb_pred = model_cb.predict(df_test)\n\nrmse\n"

In [43]:
'''
# XGBoostによる予想

xgb_train = xgb.DMatrix(train_x, label=train_y)
xgb_val = xgb.DMatrix(val_x, label=val_y)
xgb_test = xgb.DMatrix(df_test)

params = {'objective':'reg:squarederror',
          'random_state':123}

model_xgb = xgb.train(params, dtrain=xgb_train, num_boost_round=1000)

val_xgb = model_xgb.predict(xgb_val)
rmse = np.sqrt(mse(val_y, val_xgb))

xgb_pred = model_xgb.predict(xgb_test)

rmse

'''

"\n# XGBoostによる予想\n\nxgb_train = xgb.DMatrix(train_x, label=train_y)\nxgb_val = xgb.DMatrix(val_x, label=val_y)\nxgb_test = xgb.DMatrix(df_test)\n\nparams = {'objective':'reg:squarederror',\n          'random_state':123}\n\nmodel_xgb = xgb.train(params, dtrain=xgb_train, num_boost_round=1000)\n\nval_xgb = model_xgb.predict(xgb_val)\nrmse = np.sqrt(mse(val_y, val_xgb))\n\nxgb_pred = model_xgb.predict(xgb_test)\n\nrmse\n\n"

In [44]:
'''
stacking_list = []
stacking_list.append(val_lgb.tolist())
stacking_list.append(val_cb.tolist())
#stacking_list.append(val_xgb.tolist())
stacking_list.append(stacking_answer)

stacking_df_t = pd.DataFrame(stacking_list, index=["lgb","cb", "answer"])

stacking_df = stacking_df_t.T
'''

'\nstacking_list = []\nstacking_list.append(val_lgb.tolist())\nstacking_list.append(val_cb.tolist())\n#stacking_list.append(val_xgb.tolist())\nstacking_list.append(stacking_answer)\n\nstacking_df_t = pd.DataFrame(stacking_list, index=["lgb","cb", "answer"])\n\nstacking_df = stacking_df_t.T\n'

In [45]:
'''
st_train, st_val = train_test_split(stacking_df, test_size=0.2, shuffle=True, random_state=123)

st_train_y = st_train["answer"]
st_train_x = st_train.drop("answer", axis=1)

st_val_y = st_val["answer"]
st_val_x = st_val.drop("answer", axis=1)
'''


'\nst_train, st_val = train_test_split(stacking_df, test_size=0.2, shuffle=True, random_state=123)\n\nst_train_y = st_train["answer"]\nst_train_x = st_train.drop("answer", axis=1)\n\nst_val_y = st_val["answer"]\nst_val_x = st_val.drop("answer", axis=1)\n'

In [46]:
'''
def objective(trial):

    st_lgb_trains = lgb.Dataset(st_train_x, st_train_y)
    st_lgb_valids = lgb.Dataset(st_val_x, st_val_y)
    
    learning_rate = trial.suggest_float('learning_rate', 0.0, 1.0)
    num_leaves =  trial.suggest_int("num_leaves", 2, 100)
    tree_learner = trial.suggest_categorical('tree_learner', ["serial", "feature", "data", "voting"])
    lambda_l1= trial.suggest_float("lambda_l1", 0.0, 200.0)
    lambda_l2 = trial.suggest_float("lambda_l2", 0.0, 200.0)
    max_depth = trial.suggest_int("max_depth", 2, 12)
    num_iterations = trial.suggest_int("num_iterations", 10, 100)

    lgb_params = {
        "objective": "regression",
        "boosting_type": "gbdt", 
        "metrics": "rmse", 
        "learning_rate": learning_rate, 
        "num_leaves": num_leaves, 
        "tree_learner": tree_learner,
        "lambda_l1": lambda_l1, 
        "lambda_l2": lambda_l2, 
        "seed": 123, 
        "max_depth": max_depth,
        "num_iterations": num_iterations      
    }

    regressor = lgb.train(lgb_params,
                        st_lgb_trains,
                        valid_sets=st_lgb_valids
                        )

    lgb_pred_st_val_y = regressor.predict(st_val_x)

    rmse = np.sqrt(mse(st_val_y, lgb_pred_st_val_y))
    
    return rmse

st_study = optuna.create_study(direction='minimize')
st_study.optimize(objective, n_trials=100)
'''

'\ndef objective(trial):\n\n    st_lgb_trains = lgb.Dataset(st_train_x, st_train_y)\n    st_lgb_valids = lgb.Dataset(st_val_x, st_val_y)\n    \n    learning_rate = trial.suggest_float(\'learning_rate\', 0.0, 1.0)\n    num_leaves =  trial.suggest_int("num_leaves", 2, 100)\n    tree_learner = trial.suggest_categorical(\'tree_learner\', ["serial", "feature", "data", "voting"])\n    lambda_l1= trial.suggest_float("lambda_l1", 0.0, 200.0)\n    lambda_l2 = trial.suggest_float("lambda_l2", 0.0, 200.0)\n    max_depth = trial.suggest_int("max_depth", 2, 12)\n    num_iterations = trial.suggest_int("num_iterations", 10, 100)\n\n    lgb_params = {\n        "objective": "regression",\n        "boosting_type": "gbdt", \n        "metrics": "rmse", \n        "learning_rate": learning_rate, \n        "num_leaves": num_leaves, \n        "tree_learner": tree_learner,\n        "lambda_l1": lambda_l1, \n        "lambda_l2": lambda_l2, \n        "seed": 123, \n        "max_depth": max_depth,\n        "num_i

In [47]:
'''
st_trains = lgb.Dataset(st_train_x, st_train_y)
st_valids = lgb.Dataset(st_val_x, st_val_y)

st_model_lgb = lgb.train(st_study.best_trial.params, 
                    st_trains, 
                    valid_sets=st_valids
                    )

st_val_lgb = st_model_lgb.predict(st_val_x)
rmse = np.sqrt(mse(st_val_y, st_val_lgb))
'''

'\nst_trains = lgb.Dataset(st_train_x, st_train_y)\nst_valids = lgb.Dataset(st_val_x, st_val_y)\n\nst_model_lgb = lgb.train(st_study.best_trial.params, \n                    st_trains, \n                    valid_sets=st_valids\n                    )\n\nst_val_lgb = st_model_lgb.predict(st_val_x)\nrmse = np.sqrt(mse(st_val_y, st_val_lgb))\n'

In [48]:
'''
test_stacking_list = []
test_stacking_list.append(lgb_pred.tolist())
test_stacking_list.append(cb_pred.tolist())
#test_stacking_list.append(xgb_pred.tolist())

test_stacking_df_t = pd.DataFrame(test_stacking_list, index=["lgb","cb"])

test_stacking_df = test_stacking_df_t.T
'''

'\ntest_stacking_list = []\ntest_stacking_list.append(lgb_pred.tolist())\ntest_stacking_list.append(cb_pred.tolist())\n#test_stacking_list.append(xgb_pred.tolist())\n\ntest_stacking_df_t = pd.DataFrame(test_stacking_list, index=["lgb","cb"])\n\ntest_stacking_df = test_stacking_df_t.T\n'

In [49]:
#final_predict = st_model_lgb.predict(test_stacking_df)

In [50]:
#test_stacking_df["final"] = final_predict

In [51]:
#test_stacking_df

In [52]:
#submit = np.exp(test_stacking_df["final"]) - 10

In [53]:
#submit.to_csv("submit19.csv", index=True, header=False)

In [54]:
#df_test["n"].to_csv("submit16.csv", index=True, header=False)

In [55]:
#lightgbmのfeature_importance関数を使って特徴量重要度を見てみる
#pd.DataFrame(model_lgbm.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)